In [1]:
import pandas as pd
import geopandas as gpd 


In [2]:
subway=pd.read_csv('./subway1.csv',encoding='utf-8') 

In [3]:
subway

,s_nm,s_long,s_lat
0,다대포해수욕장(1호선),128.965919,35.048429
1,다대포항(1호선),128.971251,35.057288
2,낫개(1호선),128.979932,35.065179
3,신장림(1호선),128.976638,35.074222
4,장림(1호선),128.977638,35.081555
...,...,...,...
123,신해운대(동해선),129.177087,35.181674
124,송정(동해선),129.199931,35.180928
125,오시리아(동해선),129.208290,35.196247
126,기장(동해선),129.218841,35.243785


In [4]:
busan_travel=pd.read_csv('./travel_list2.csv',encoding='utf-8') # 관광지정보read
busan_travel

,travel_spot_name,ts_long,ts_lat
0,서면1번가,129.058286,35.153576
1,재난안전체험관,129.105109,35.291822
2,광안리해수욕장,129.118492,35.153791
3,송정해수욕장,129.199640,35.178712
4,해운대해수욕장,129.160308,35.159124
...,...,...,...
559,연제고분판타지축제,129.095056,35.192581
560,다대포어항문화축제,128.972821,35.055623
561,보수동책방골목문화축제,129.027461,35.103125
562,부산고등어축제,129.016761,35.075455


In [5]:
busan_subway_gdf = gpd.GeoDataFrame(subway,geometry=gpd.points_from_xy(subway.s_long,subway.s_lat),crs='epsg:4326')  # WGS84(EPSG:4326) :경도위도좌표계 
busan_travel_gdf = gpd.GeoDataFrame(busan_travel,geometry=gpd.points_from_xy(busan_travel.ts_long,busan_travel.ts_lat),crs='epsg:4326')  

# 부산버스지오df 

In [6]:
busan_travel_gdf

,travel_spot_name,ts_long,ts_lat,geometry
0,서면1번가,129.058286,35.153576,POINT (129.05829 35.15358)
1,재난안전체험관,129.105109,35.291822,POINT (129.10511 35.29182)
2,광안리해수욕장,129.118492,35.153791,POINT (129.11849 35.15379)
3,송정해수욕장,129.199640,35.178712,POINT (129.19964 35.17871)
4,해운대해수욕장,129.160308,35.159124,POINT (129.16031 35.15912)
...,...,...,...,...
559,연제고분판타지축제,129.095056,35.192581,POINT (129.09506 35.19258)
560,다대포어항문화축제,128.972821,35.055623,POINT (128.97282 35.05562)
561,보수동책방골목문화축제,129.027461,35.103125,POINT (129.02746 35.10313)
562,부산고등어축제,129.016761,35.075455,POINT (129.01676 35.07545)


In [8]:
subway_distances = busan_subway_gdf.to_crs(epsg=5186).geometry.distance(busan_travel_gdf.to_crs(epsg=5186).iloc[0].geometry)

In [9]:
subway_distances

0      14393.551603
1      13312.664849
2      12137.809101
3      11533.321522
4      10862.271667
           ...     
123    11268.085263
124    13262.259256
125    14467.924735
126    17726.738649
127    20215.288560
Length: 128, dtype: float64

In [12]:
near_subway = subway.iloc[subway_distances.idxmin()]['s_nm']
print(f'{busan_travel_gdf.iloc[0].travel_spot_name}에서 가장가까운 지하철역: {near_subway} ({subway_distances.min()}m)')
# 서면1번가에서 가장가까운 버스정류장: 서면1번가 (37.32908144279374m)


서면1번가에서 가장가까운 지하철역: 서면(1호선) (473.5242650819915m)


In [14]:
def closet_subway(point_geometry):
    distances =  busan_subway_gdf.to_crs(epsg=5186).geometry.distance(point_geometry)
    return subway.iloc[distances.idxmin()]['s_nm'] # idxmin : 최솟값 인덱스찾기

busan_travel_gdf['travel_subway'] = busan_travel_gdf.to_crs(epsg=5186).apply(lambda x: closet_subway(x.geometry), axis=1)

In [15]:
busan_travel_gdf

,travel_spot_name,ts_long,ts_lat,geometry,travel_subway
0,서면1번가,129.058286,35.153576,POINT (129.05829 35.15358),서면(1호선)
1,재난안전체험관,129.105109,35.291822,POINT (129.10511 35.29182),노포(1호선)
2,광안리해수욕장,129.118492,35.153791,POINT (129.11849 35.15379),광안(2호선)
3,송정해수욕장,129.199640,35.178712,POINT (129.19964 35.17871),송정(동해선)
4,해운대해수욕장,129.160308,35.159124,POINT (129.16031 35.15912),중동(2호선)
...,...,...,...,...,...
559,연제고분판타지축제,129.095056,35.192581,POINT (129.09506 35.19258),동래(동해선)
560,다대포어항문화축제,128.972821,35.055623,POINT (128.97282 35.05562),다대포항(1호선)
561,보수동책방골목문화축제,129.027461,35.103125,POINT (129.02746 35.10313),자갈치(1호선)
562,부산고등어축제,129.016761,35.075455,POINT (129.01676 35.07545),자갈치(1호선)


In [16]:
busan_travel_gdf.to_csv('./travel_subway.csv',encoding='utf-8')